https://adventofcode.com/2022/day/11

trying with classes. . . 

Monkey#
    Has items (list) (of WL number)
    Has operation (fn) (on WLN )
    Has test (fn,target)

M:
1. inspects item
1. worry level / 3 round down to integer
    * import math
    * WL = math.floor(WL/3)

Monkeys go in numerical order:
1. inspect each item in list order
1. throws item to other monkey
    * item added to end of target M list of items



In [99]:
import re
import operator
import copy


class Monkey:
    def __init__(self, monkey_id:int, items:list, worry_op:str, div:int, throw_to:list) -> None:
        self.monkey_id = monkey_id
        self.start_items =  items
        self._worry_op = worry_op # operation on worry number ( * / + Num)
        self.divisor = div # test division
        self._throw_to = throw_to # test target
        self.inspection_count = 0 # how many loops to go through?

    def add_item(self, item:int):
        self.start_items.append(item)
    
    def inspect(self) -> int: # after updating all the internal variables, returns the monkey # to throw item to
        self.inspection_count += 1
        self._worry_op.replace('old',str(self.start_items[0])) # _worry_op has the string from the input after '=':  'old * N'
            # so this replaces the word "old" with the current value in the worry list: "32 * N"
        first, the_op, second = re.findall(r"(\w+) (.) (\w+)",worry_op)[0]
            # search the worry_op string (32 * N). Findall returns a list [(32,*,N)] of tuples
            # list[0] returns the tuple, which is assigned to the variables
        ops_dict = {
            '+' : operator.add,
            '*' : operator.mul
        }
            # https://docs.python.org/3/library/operator.html
            # operator.XXX is list of functions in std library
            # operator.add(x,y) is the same as x+y

        self.start_items[0] = ops_dict[the_op](int(first),int(second))
            # replace the first WL item with the value of the equation
            # item[0] = operator.add(firstVal,secondVal)

        self.start_items[0] //= 3 # divide by 3 and round down

        return self._throw_to[0] if self.start_items[0] % self.divisor == 0 \
                                 else self._throw_to[1]
                                 
    def throw_to(self,other: Monkey):
        other.add_item(self.start_items.pop(0))
            # .pop removes item from list at given index and RETURNS the item
            # so this takes item[0] from self and sends it to other
            # other adds the item using fn above (take item and append to end of list)

    def __repr__(self) -> str:
        return f"Monkey:(id={self.monkey_id}, items={self.start_items}, " \
                + f"inspect_count=({self.inspection_count})"
      


In [134]:
class Monkey:
    """ The monkey has a bunch of my things. 
    start_items = worry level for each item, in the order they will be inspectd
    worry_op = how worry level changes as the monkey inspects the item
    """
    def __init__(self, monkey_id: int, items: list, worry_op: str, div: int, throw_to: list) -> None:
        self.monkey_id = monkey_id # E.g. 0
        self.start_items = items # E.g. [79, 98]
        self._worry_op = worry_op  # E.g. old * 19
        self.divisor = div  # E.g. 13
        self._throw_to = throw_to # E.g. [2, 3]
        self.inspect_count = 0
    
    def add_item(self, item:int):
        self.start_items.append(item)
        
    def inspect(self) -> int:
        """ Inspects the next item in the list. 
        Inspecting causes our worry level to go up, as given by worry_op. 
        If relief enabled, we then reduce our worry level by //3.
        Then we work out who to throw to, by dividing by a divisor.
        
        In part 2:
          - relief is disabled and worry level can get VERY LARGE!!
          - We can significantly reduce this number by using LCM trick.
         """
        
        self.inspect_count += 1
        
        # turn "old * 19" into "79 * 19"
        worry_op = self._worry_op.replace("old", str(self.start_items[0]))
        
        first, the_op, second = re.findall(r"(\w+) (.) (\w+)", worry_op)[0]
        ops_dict = {
            "+": operator.add,
            "*": operator.mul
        }
        
        self.start_items[0] = ops_dict[the_op](int(first), int(second))
    
        # Relief. Rule = divide by three and round down
        self.start_items[0] //= 3
        
        return self._throw_to[0] if self.start_items[0] % self.divisor == 0 \
                                 else self._throw_to[1]
    
    def throw_to(self, other: Monkey):
        other.add_item(self.start_items.pop(0))
        
    def __repr__(self) -> str:
        return f"Monkey:(id={self.monkey_id}, items={self.start_items}, " \
                + f"inspect_count={self.inspect_count})"


In [135]:
def parse_input(data:str) -> dict[int,Monkey]:
    blocks = data.split('\n\n')

    monkeys = {}
    for block in blocks:
        #print(block)
        for line in block.splitlines():
            if line.startswith("Monkey"):
                monkey_id = int(re.findall(r"(\d+)", line)[0])
                
            if "items:" in line:
                items = list(map(int, re.findall(r"(\d+)", line)))
                    # map(fn,iterable) applies the function to each value in the iterable
                    # returns the results
                    # list() converts the results into a list

            if "Operation:" in line:
                worry_op = line.split("=")[-1].strip()
                    # split the line at '=' and return second half ([-1])
                    # new = old * 3 --> 'old * 3'
            
            if "Test:" in line:
                divisor = int(re.findall(r"\d+", line)[0])
            
            if "true:" in line:
                to_monkey_true = int(re.findall(r"\d+", line)[0])
            
            if "false:" in line:
                to_monkey_false = int(re.findall(r"\d+", line)[0])

        #print(monkey_id, items, worry_op, divisor, to_monkey_true, to_monkey_false)

        monkey = Monkey(monkey_id = monkey_id,
                        items = items, 
                        worry_op = worry_op,
                        div = divisor,
                        throw_to = [to_monkey_true,to_monkey_false]
                        )
            # create a monkey instance

        monkeys[monkey_id] = monkey
            # add the monkey instance to the dictionary; key = id num, val = monkey instance
    #for i in monkeys.values():
    #    print(i)
    
    return monkeys # return dictionary




In [136]:
data = """Monkey 0:
  Starting items: 75, 63
  Operation: new = old * 3
  Test: divisible by 11
    If true: throw to monkey 7
    If false: throw to monkey 2

Monkey 1:
  Starting items: 65, 79, 98, 77, 56, 54, 83, 94
  Operation: new = old + 3
  Test: divisible by 2
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 66
  Operation: new = old + 5
  Test: divisible by 5
    If true: throw to monkey 7
    If false: throw to monkey 5

Monkey 3:
  Starting items: 51, 89, 90
  Operation: new = old * 19
  Test: divisible by 7
    If true: throw to monkey 6
    If false: throw to monkey 4

Monkey 4:
  Starting items: 75, 94, 66, 90, 77, 82, 61
  Operation: new = old + 1
  Test: divisible by 17
    If true: throw to monkey 6
    If false: throw to monkey 1

Monkey 5:
  Starting items: 53, 76, 59, 92, 95
  Operation: new = old + 2
  Test: divisible by 19
    If true: throw to monkey 4
    If false: throw to monkey 3

Monkey 6:
  Starting items: 81, 61, 75, 89, 70, 92
  Operation: new = old * old
  Test: divisible by 3
    If true: throw to monkey 0
    If false: throw to monkey 1

Monkey 7:
  Starting items: 81, 86, 62, 87
  Operation: new = old + 8
  Test: divisible by 13
    If true: throw to monkey 3
    If false: throw to monkey 5
    """

In [141]:
monkeys = parse_input(data)

In [140]:
from collections import Counter

def play(monkeys: dict[int, Monkey], rounds_to_play: int) -> int:
    """ Play required number of rounds.
    Returns 'Monkey Business' = product of the top two inspection counts """
    for _ in range(1, rounds_to_play+1):
        for monkey in monkeys.values(): # Iterator through monkeys in order
            while monkey.start_items: # Monkey inspects and throws until it has no more items
                to_monkey = monkeys[monkey.inspect()]
                monkey.throw_to(to_monkey)
    
    # Get the two monkeys that have inspected the most
    monkey_inspect = Counter({monkey.monkey_id: monkey.inspect_count for monkey in monkeys.values()})
    two_most_common = monkey_inspect.most_common(2)
    return two_most_common[0][1] * two_most_common[1][1]


In [142]:
play(monkeys,20)

62491

In [143]:
for i in monkeys.values():
    print(i)

Monkey:(id=0, items=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], inspect_count=194)
Monkey:(id=1, items=[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], inspect_count=242)
Monkey:(id=2, items=[], inspect_count=240)
Monkey:(id=3, items=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 12], inspect_count=243)
Monkey:(id=4, items=[], inspect_count=247)
Monkey:(id=5, items=[], inspect_count=253)
Monkey:(id=6, items=[], inspect_count=17)
Monkey:(id=7, items=[], inspect_count=17)


submitted 129600 - incorrect, too high.
submitted 32,400 -- incorrect too low. (I just re-ran it without doing any inspections / calls on moneky, since that affects what is in the lists. . . )



In [86]:
monkeys[0].inspect()

7

In [15]:
inputT = """Monkey 0:
  Starting items: 75, 63
  Operation: new = old * 3
  Test: divisible by 11
    If true: throw to monkey 7
    If false: throw to monkey 2

Monkey 1:
  Starting items: 65, 79, 98, 77, 56, 54, 83, 94
  Operation: new = old + 3
  Test: divisible by 2
    If true: throw to monkey 2
    If false: throw to monkey 0"""

In [34]:
re.findall(r"(\d+)", "Starting items: 65, 79, 98, 77, 56, 54, 83, 94")

['65', '79', '98', '77', '56', '54', '83', '94']

In [37]:
map(int,re.findall(r"(\d+)", "Starting items: 65, 79, 98, 77, 56, 54, 83, 94"))


In [3]:
inputGroups = inputT.split('\n\n')

In [12]:
import re
input_data = []

for blob in inputGroups:
    monkeyNum = re.findall(r"Monkey (\d)",blob)[0]
    temp = re.findall(r"new = old (\S) (\d)",blob)
    op_vals = (temp[0][0],int(temp[0][1]))
    #op_str = f"x {opvals[0][0]}= {opvals[0][1]}"
    testDiv = re.findall(r"by (\d+)",blob)[0]
    testTrueTarg = re.findall(r"true: throw to monkey (\d+)",blob)[0]
    testFalseTarg = re.findall(r"false: throw to monkey (\d+)",blob)[0]
    test_vals = (int(testDiv),testTrueTarg,testFalseTarg)
    #test_str = f"if (math.floor(numb / 3) % {testDiv}: == 0: throwTo {testTrueTarg} else throwTo {testFalseTarg}"
    input_data.append([monkeyNum,op_vals,test_vals])

input_data

[['0', ('*', 3), (11, '7', '2')], ['1', ('+', 3), (2, '2', '0')]]

In [25]:
worry_op = " Operation: new = old + 3".split("=")[-1].strip()
worry_op

'old + 3'

In [32]:
worry_op = worry_op.replace('old',str('32'))
print(worry_op)
re.findall(r"(\w+) (.) (\w+)",worry_op)[0]


32 + 3


('32', '+', '3')

In [144]:
12%5

2

In [145]:
18%5

3

In [151]:
1 % 10 == 101 % 10

True